In [1]:
# For Data Processing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
import plotly.express as px
from imblearn.over_sampling import SMOTE, RandomOverSampler
import torch
import torch.nn as nn
import torchbnn as bnn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# Import ML Model Classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

# For Model Evaluation
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import roc_curve, auc 
from matplotlib import pyplot

In [21]:
colectomy_cci_var = pd.read_csv('./colectomy_cci_var.csv', dtype = 'str')

colectomy_cci_var = colectomy_cci_var.groupby('pdx').filter(lambda x: len(x) >= 1000)

for feature in colectomy_cci_var.columns:
    encoder = LabelEncoder()
    colectomy_cci_var[feature] = encoder.fit_transform(colectomy_cci_var[feature])
    colectomy_cci_var[feature] = colectomy_cci_var[feature].astype(int)

colectomy_cci_var_y = colectomy_cci_var['pdx']
colectomy_cci_var_X = colectomy_cci_var.drop('pdx', axis = 1)


# Split Data
train_X, test_X, train_y, test_y = train_test_split(colectomy_cci_var_X, colectomy_cci_var_y, test_size = 0.2, random_state = 0)

# Data Resampling
smote = RandomOverSampler(sampling_strategy = 'auto', random_state = 0)
train_X, train_y = smote.fit_resample(train_X, train_y)

# Frequency encoding
for feature in train_X.columns:
    category_counts = train_X[feature].value_counts()
    train_X[feature] = train_X[feature].map(category_counts)

    category_counts = test_X[feature].value_counts()
    test_X[feature] = test_X[feature].map(category_counts)

# Label Encoding
encoder = LabelEncoder()
label_encoder = encoder.fit(train_y)
train_y = label_encoder.transform(train_y)
test_y = label_encoder.transform(test_y)

scaler = MinMaxScaler()

train_X_normalized = train_X.copy()
test_X_normalzied = test_X.copy()

normalize_features = train_X.columns

normalizer = scaler.fit(train_X_normalized[normalize_features])

train_X_normalized[normalize_features] = normalizer.transform(train_X_normalized[normalize_features])
test_X_normalzied[normalize_features] = normalizer.transform(test_X_normalzied[normalize_features])

X_train = train_X_normalized
X_test = test_X_normalzied
Y_train = train_y
Y_test = test_y

In [ ]:
clf_rand_forest = RandomForestClassifier()
clf_rand_forest.fit(X_train, Y_train)

importances = clf_rand_forest.feature_importances_

# Sort and display feature importances
sorted_idx = importances.argsort()
for idx in sorted_idx:
    print(f"Feature: {X_train.columns[idx]}, Importance: {importances[idx]}")

RandomForestClassifier()

In [3]:
colectomy_cci_var = pd.read_csv('./colectomy_cci_var.csv', dtype = 'str')

# Set random seed
np.random.seed(42)

# Rename columns
colectomy_cci_var['ptsex'] = colectomy_cci_var['ptsex'].replace({'F': 'female', 'M': 'male'})
colectomy_cci_var['adtype'] = colectomy_cci_var['adtype'].replace({'1': 'emergency', '2': 'urgent', '3': 'elective'})
colectomy_cci_var['adsource'] = colectomy_cci_var['adsource'].replace({'1': 'non_healthcare_facility', '2': 'clinical_referral', '4': 'hospital_transfer', '5': 'snf_icf_transfer', '6': 'healthcare_facility_tranfer', '7': 'emergency_room', '8': 'court_law_enforcement', 'D': 'hospital_unit_transfer', 'E': 'ambulatory_surgery_center'})

colectomy_cci_var['age'] = colectomy_cci_var['age'].astype(int)

colectomy_cci_var['paytype1'] = colectomy_cci_var['paytype1'].fillna('99')

# Drop observations
colectomy_cci_var = colectomy_cci_var[colectomy_cci_var['adsource'] != '9']
colectomy_cci_var = colectomy_cci_var[colectomy_cci_var['adsource'] != 'emergency_room']
colectomy_cci_var = colectomy_cci_var[colectomy_cci_var['adsource'] != 'court_law_enforcement']
colectomy_cci_var = colectomy_cci_var[colectomy_cci_var['adsource'] != 'hospital_unit_transfer']

#print(len(colectomy_cci_var))
colectomy_cci_var = colectomy_cci_var.groupby('pdx').filter(lambda x: len(x) >= 1000)
#print(len(colectomy_cci_var))

# Select Relevant Features
features = ['year', 'ptsex', 'race', 'age', 'adtype', 'adsource', 'admdx']
dummy_features = ['year', 'ptsex', 'race', 'adtype', 'adsource']
frequency_features = ['admdx']

# Data Preprocessing
num_secondary_admissions = 1

for i in range(1, num_secondary_admissions + 1):
    sdx = f'sdx{i}'

    features.append(sdx)
    frequency_features.append(sdx)

    # Replace missing values with the "unknown" category
    colectomy_cci_var[sdx] = colectomy_cci_var[sdx].fillna('unknown')
    #colectomy_cci_var[sdx] = colectomy_cci_var[sdx].fillna(-1)
    '''
    # Frequency encoding
    category_counts = colectomy_cci_var[sdx].value_counts()
    colectomy_cci_var[sdx] = colectomy_cci_var[sdx].map(category_counts)
    '''

'''
# Frequency encoding
category_counts = colectomy_cci_var['admdx'].value_counts()
colectomy_cci_var['admdx'] = colectomy_cci_var['admdx'].map(category_counts)
'''

for feature in frequency_features:
    encoder = LabelEncoder()
    colectomy_cci_var[feature] = encoder.fit_transform(colectomy_cci_var[feature])
    colectomy_cci_var[feature] = colectomy_cci_var[feature].astype(int)

#for feature in features:
#    print(colectomy_cci_var[feature].value_counts())

colectomy_cci_var_X = colectomy_cci_var[features]
colectomy_cci_var_y = colectomy_cci_var['pdx']

# One-Hot Encoding
colectomy_cci_var_X = pd.get_dummies(colectomy_cci_var_X, columns = dummy_features, dtype = 'int')

# Split Data
train_X, test_X, train_y, test_y = train_test_split(colectomy_cci_var_X, colectomy_cci_var_y, test_size = 0.2, random_state = 0)

# Data Resampling
smote = SMOTE(sampling_strategy = 'auto', random_state = 0)
train_X, train_y = smote.fit_resample(train_X, train_y)

#oversampler = RandomOverSampler(random_state = 0)
#train_X, train_y = oversampler.fit_resample(train_X, train_y)

# Frequency encoding
for feature in frequency_features:
    category_counts = train_X[feature].value_counts()
    train_X[feature] = train_X[feature].map(category_counts)

    category_counts = test_X[feature].value_counts()
    test_X[feature] = test_X[feature].map(category_counts)

# Label Encoding
encoder = LabelEncoder()
label_encoder = encoder.fit(train_y)
train_y = label_encoder.transform(train_y)
test_y = label_encoder.transform(test_y)

# Normalize Data
normalize_features = list(set(features) - set(dummy_features))

scaler = MinMaxScaler()

train_X_normalized = train_X.copy()
test_X_normalzied = test_X.copy()

normalizer = scaler.fit(train_X_normalized[normalize_features])

train_X_normalized[normalize_features] = normalizer.transform(train_X_normalized[normalize_features])
test_X_normalzied[normalize_features] = normalizer.transform(test_X_normalzied[normalize_features])

X_train = train_X_normalized
X_test = test_X_normalzied
Y_train = train_y
Y_test = test_y

In [ ]:
clf_rand_forest = RandomForestClassifier()
clf_rand_forest.fit(X_train, Y_train)

importances = clf_rand_forest.feature_importances_

# Sort and display feature importances
sorted_idx = importances.argsort()
for idx in sorted_idx:
    print(f"Feature: {X_train.columns[idx]}, Importance: {importances[idx]}")

In [8]:
# Model Architecture
class BayesianNeuralNetwork(nn.Module):
    def __init__(self, feature_size, num_classes):
        super().__init__()

        layer_1_size = 16
        layer_2_size = 32
        layer_3_size = 16
        layer_4_size = 16

        prior_mean = 0
        prior_variance = 1

        # Layers
        self.bayes_linear_1 = bnn.BayesLinear(prior_mu = prior_mean, prior_sigma = prior_variance, in_features = feature_size, out_features = layer_1_size)
        self.bayes_linear_2 = bnn.BayesLinear(prior_mu = prior_mean, prior_sigma = prior_variance, in_features = layer_1_size, out_features = layer_2_size)
        #self.bayes_linear_3 = bnn.BayesLinear(prior_mu = prior_mean, prior_sigma = prior_variance, in_features = layer_2_size, out_features = layer_3_size)
        #self.bayes_linear_4 = bnn.BayesLinear(prior_mu = prior_mean, prior_sigma = prior_variance, in_features = layer_3_size, out_features = layer_4_size)
        self.output_layer = nn.Linear(in_features = layer_2_size, out_features = num_classes)
    
    def forward(self, x):
        x = torch.relu(self.bayes_linear_1(x))
        x = torch.relu(self.bayes_linear_2(x))
        #x = torch.relu(self.bayes_linear_3(x))
        #x = torch.relu(self.bayes_linear_4(x))
        logits = self.output_layer(x)
        #probabilities = torch.softmax(logits, dim=-1)
        return logits
    
# Loss Function
def elbo_loss(output, target, model):
    standard_loss = nn.CrossEntropyLoss()(output, target)
    # The BKLLoss function automatically predfines the variational distribution as a normal distribution
    kl_loss = bnn.BKLLoss(reduction = 'mean', last_layer_only = False)

    return standard_loss + kl_loss(model)

def train(model, train_loader, optimizer, epochs = 5):
    model.train()
    for epoch in range(epochs):
        for data, target in train_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = elbo_loss(output, target, model)
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch + 1}, Loss: {loss.item()}")
        
        '''
        # After each epoch, evaluate on the validation set
        model.eval()  # Set model to evaluation mode
        val_loss = 0.0
        with torch.no_grad():
            for data, target in val_loader:
                output = model(data)
                val_loss += elbo_loss(output, target, model).item()

        # Calculate average validation loss
        val_loss /= len(val_loader)

        print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}")

        # Save the model if it has improved
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict()  # Save model parameters
            best_posterior = model
        '''

best_val_loss = float('inf')  # Initialize best validation loss to a high value
best_model_state = None  # To store the best model's state_dict
best_posterior = None

X_train_tensor = torch.tensor(X_train.values).float()
Y_train_tensor = torch.tensor(pd.Series(Y_train).values).long()

dataset = TensorDataset(X_train_tensor, Y_train_tensor)
train_loader = DataLoader(dataset, batch_size = 32, shuffle = True)

model = BayesianNeuralNetwork(feature_size = X_train.shape[1], num_classes = len(torch.unique(Y_train_tensor)))
optimizer = optim.Adam(model.parameters(), lr = 0.001)
train(model, train_loader, optimizer, 10)

Epoch 1, Loss: 1.8517273664474487
Epoch 2, Loss: 1.8527415990829468
Epoch 3, Loss: 1.768592119216919
Epoch 4, Loss: 1.810803771018982
Epoch 5, Loss: 1.7685190439224243
Epoch 6, Loss: 1.683427095413208
Epoch 7, Loss: 1.7932090759277344
Epoch 8, Loss: 1.734788179397583
Epoch 9, Loss: 2.0626344680786133
Epoch 10, Loss: 1.5346059799194336


In [9]:
X_test_tensor = torch.tensor(X_test.values).float()

# Sampling from posterior (to account for uncertainty in predictions)
num_samples = 100  # Number of posterior samples

model.eval()  # Set model to evaluation mode
outputs = []

with torch.no_grad():
    for i in range(num_samples):
        output_sample = model(X_test_tensor) # Forward pass with sampled weights
        outputs.append(output_sample)

# Convert outputs to numpy for easier processing
outputs = torch.stack(outputs).numpy()  # Shape: (num_samples, num_test_samples, num_classes)
# Get the mean output across posterior samples
mean_output = np.mean(outputs, axis = 0)  # Shape: (num_test_samples, num_classes)

# Get predicted class (for classification, pick the class with the highest probability)
predicted_classes = np.argmax(mean_output, axis = 1)

# Evaluate the performance using sklearn metrics
accuracy = accuracy_score(Y_test, predicted_classes)
report = classification_report(Y_test, predicted_classes, zero_division = 1)

print(f"Accuracy: {accuracy:.4f}")
print(f"Classification Report:\n{report}")

Accuracy: 0.1120
Classification Report:
              precision    recall  f1-score   support

           0       0.13      0.19      0.15       240
           1       0.09      0.17      0.11       462
           2       0.12      0.67      0.20       454
           3       1.00      0.00      0.00       564
           4       0.10      0.06      0.08       740
           5       1.00      0.00      0.00      1774

    accuracy                           0.11      4234
   macro avg       0.41      0.18      0.09      4234
weighted avg       0.60      0.11      0.06      4234



In [10]:
X_train

,age,admdx,sdx1,year_2011,year_2012,year_2013,year_2014,ptsex_female,ptsex_male,race_A,...,race_W,adtype_elective,adtype_emergency,adtype_urgent,adsource_ambulatory_surgery_center,adsource_clinical_referral,adsource_healthcare_facility_tranfer,adsource_hospital_transfer,adsource_non_healthcare_facility,adsource_snf_icf_transfer
0,0.487805,0.283092,0.015308,0,0,0,1,0,1,0,...,1,1,0,0,0,0,0,0,1,0
1,0.292683,0.819555,0.231399,0,0,1,0,0,1,0,...,1,1,0,0,0,0,0,0,1,0
2,0.792683,1.000000,0.002848,0,1,0,0,0,1,0,...,1,1,0,0,0,0,0,0,1,0
3,0.536585,0.819555,0.357067,1,0,0,0,0,1,0,...,1,1,0,0,0,0,0,0,1,0
4,0.378049,0.051161,0.028836,0,1,0,0,1,0,0,...,1,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41965,0.487805,1.000000,0.184763,0,0,0,1,1,0,0,...,1,1,0,0,0,0,0,0,0,0
41966,0.621951,1.000000,0.150943,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
41967,0.463415,1.000000,0.048416,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0
41968,0.463415,0.006172,0.009612,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
